### 1.准备数据集

In [1]:
!python sual/tools/prepare_dataset/prepare_active_dataset.py \
    data/ForestDamages \
    data/ForestDamages/active_learning_cascade_ssc \
    --train-ratio 0.04 \
    --val-ratio 0.01 \
    --seed 42
# 批量运行  
# ./custom_config/prepare_experiments.sh


合并数据集...
数据根目录: /data/22_huangguobin/mmdetection-3.1.0/data/ForestDamages
训练集标注文件: /data/22_huangguobin/mmdetection-3.1.0/data/ForestDamages/annotations/instances_train2024.json
验证集标注文件: /data/22_huangguobin/mmdetection-3.1.0/data/ForestDamages/annotations/instances_val2024.json
训练集图片目录: /data/22_huangguobin/mmdetection-3.1.0/data/ForestDamages/train2024
验证集图片目录: /data/22_huangguobin/mmdetection-3.1.0/data/ForestDamages/val2024
合并后总图像数: 3170
合并后总标注数: 194287

数据集分割:
总样本数: 3170
训练集大小: 126 (4.0%)
验证集大小: 31 (1.0%)
未标注集大小: 3013 (95.0%)

处理 labeled_train 数据集...
从以下目录复制图片:
  - /data/22_huangguobin/mmdetection-3.1.0/data/ForestDamages/train2024
  - /data/22_huangguobin/mmdetection-3.1.0/data/ForestDamages/val2024
成功复制 126 张图片到 data/ForestDamages/active_learning_cascade_ssc/images_labeled_train
保存标注到: data/ForestDamages/active_learning_cascade_ssc/annotations/instances_labeled_train.json
图像数量: 126
标注数量: 7836

处理 labeled_val 数据集...
从以下目录复制图片:
  - /data/22_huangguobin/mmdetection-3.1.0/data/Fores

In [4]:
!python sual/tools/prepare_dataset/prepare_active_dataset.py \
    data/Bamberg_coco1024 \
    data/Bamberg_coco1024/active_learning_detr_ssc \
    --train-ratio 0.04 \
    --val-ratio 0.01 \
    --seed 42
# 批量运行  
# ./custom_config/prepare_experiments.sh


合并数据集...
数据根目录: /data/22_huangguobin/mmdetection-3.1.0/data/Bamberg_coco1024
训练集标注文件: /data/22_huangguobin/mmdetection-3.1.0/data/Bamberg_coco1024/annotations/instances_train2024.json
验证集标注文件: /data/22_huangguobin/mmdetection-3.1.0/data/Bamberg_coco1024/annotations/instances_val2024.json
训练集图片目录: /data/22_huangguobin/mmdetection-3.1.0/data/Bamberg_coco1024/train2024
验证集图片目录: /data/22_huangguobin/mmdetection-3.1.0/data/Bamberg_coco1024/val2024
Traceback (most recent call last):
  File "/data/22_huangguobin/mmdetection-3.1.0/sual/tools/prepare_dataset/prepare_active_dataset.py", line 294, in <module>
    prepare_active_dataset(
  File "/data/22_huangguobin/mmdetection-3.1.0/sual/tools/prepare_dataset/prepare_active_dataset.py", line 252, in prepare_active_dataset
    merged_data, src_img_dirs = merge_coco_datasets(data_root)
  File "/data/22_huangguobin/mmdetection-3.1.0/sual/tools/prepare_dataset/prepare_active_dataset.py", line 87, in merge_coco_datasets
    raise FileNotFoundError(f"找

In [11]:
!python tools/test.py \
    work_dirs/cascade-rcnn_ssc_16_200_weight_chage/round_1/cascade-rcnn_ssc.py \
    work_dirs/cascade-rcnn_ssc_16_200_weight_chage/round_1/epoch_3.pth \
    --work-dir work_dirs/test 


05/26 14:08:20 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.9.20 (main, Oct  3 2024, 07:27:41) [GCC 11.2.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 2126278232
    GPU 0: NVIDIA RTX A6000
    CUDA_HOME: :/usr/local/cuda
    GCC: gcc (Ubuntu 13.3.0-6ubuntu2~24.04) 13.3.0
    PyTorch: 2.0.1+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,co

#### 2.准备配置文件

In [1]:
from mmengine import Config

# 加载配置文件
cfg = Config.fromfile('configs/fcos/fcos_r101-caffe_fpn_gn-head-1x_coco.py')

# 打印完整配置
print(cfg.pretty_text)

# 保存到文件
with open('fcos.py', 'w') as f:
    f.write(cfg.pretty_text)

auto_scale_lr = dict(base_batch_size=16, enable=False)
backend_args = None
data_root = 'data/coco/'
dataset_type = 'CocoDataset'
default_hooks = dict(
    checkpoint=dict(interval=1, type='CheckpointHook'),
    logger=dict(interval=50, type='LoggerHook'),
    param_scheduler=dict(type='ParamSchedulerHook'),
    sampler_seed=dict(type='DistSamplerSeedHook'),
    timer=dict(type='IterTimerHook'),
    visualization=dict(type='DetVisualizationHook'))
default_scope = 'mmdet'
env_cfg = dict(
    cudnn_benchmark=False,
    dist_cfg=dict(backend='nccl'),
    mp_cfg=dict(mp_start_method='fork', opencv_num_threads=0))
load_from = None
log_level = 'INFO'
log_processor = dict(by_epoch=True, type='LogProcessor', window_size=50)
model = dict(
    backbone=dict(
        depth=101,
        frozen_stages=1,
        init_cfg=dict(
            checkpoint='open-mmlab://detectron/resnet101_caffe',
            type='Pretrained'),
        norm_cfg=dict(requires_grad=False, type='BN'),
        norm_eval=True,

In [4]:
from mmengine import Config

# 加载配置文件
cfg = Config.fromfile('configs/cascade_rcnn/cascade-rcnn_r101_fpn_1x_coco.py')

# 打印完整配置
print(cfg.pretty_text)

# 保存到文件
with open('cascade-rcnn.py', 'w') as f:
    f.write(cfg.pretty_text)

auto_scale_lr = dict(base_batch_size=16, enable=False)
backend_args = None
data_root = 'data/coco/'
dataset_type = 'CocoDataset'
default_hooks = dict(
    checkpoint=dict(interval=1, type='CheckpointHook'),
    logger=dict(interval=50, type='LoggerHook'),
    param_scheduler=dict(type='ParamSchedulerHook'),
    sampler_seed=dict(type='DistSamplerSeedHook'),
    timer=dict(type='IterTimerHook'),
    visualization=dict(type='DetVisualizationHook'))
default_scope = 'mmdet'
env_cfg = dict(
    cudnn_benchmark=False,
    dist_cfg=dict(backend='nccl'),
    mp_cfg=dict(mp_start_method='fork', opencv_num_threads=0))
load_from = None
log_level = 'INFO'
log_processor = dict(by_epoch=True, type='LogProcessor', window_size=50)
model = dict(
    backbone=dict(
        depth=101,
        frozen_stages=1,
        init_cfg=dict(checkpoint='torchvision://resnet101', type='Pretrained'),
        norm_cfg=dict(requires_grad=True, type='BN'),
        norm_eval=True,
        num_stages=4,
        out_indice

#### 3.各轮teacher模型推理验证集图片

In [1]:

import os
# 遍历work_dirs下的所有目录
work_dirs = os.listdir("work_dirs")
for work_dir in work_dirs:
    if work_dir.startswith("al_faster-rcnn_ssc_") or work_dir.startswith("faster-rcnn_"):
        inference_results_dir = f"work_dirs/{work_dir}/inference_results"
        print(f"\n处理目录: {inference_results_dir}")

        if os.path.exists(inference_results_dir):
            print(f"推理结果目录存在: {inference_results_dir}")
            os.system(f"rm -rf {inference_results_dir}")
            print(f"推理结果目录已删除: {inference_results_dir}")
        else:
            print(f"推理结果目录不存在: {inference_results_dir}")





处理目录: work_dirs/al_faster-rcnn_ssc_rl_kl/inference_results
推理结果目录不存在: work_dirs/al_faster-rcnn_ssc_rl_kl/inference_results

处理目录: work_dirs/al_faster-rcnn_ssc_rl_wasserstein/inference_results
推理结果目录不存在: work_dirs/al_faster-rcnn_ssc_rl_wasserstein/inference_results

处理目录: work_dirs/al_faster-rcnn_ssc_wasserstein/inference_results
推理结果目录不存在: work_dirs/al_faster-rcnn_ssc_wasserstein/inference_results

处理目录: work_dirs/al_faster-rcnn_ssc_combinatorial/inference_results
推理结果目录存在: work_dirs/al_faster-rcnn_ssc_combinatorial/inference_results
推理结果目录已删除: work_dirs/al_faster-rcnn_ssc_combinatorial/inference_results

处理目录: work_dirs/al_faster-rcnn_ssc_default/inference_results
推理结果目录存在: work_dirs/al_faster-rcnn_ssc_default/inference_results
推理结果目录已删除: work_dirs/al_faster-rcnn_ssc_default/inference_results

处理目录: work_dirs/faster-rcnn_margin_default/inference_results
推理结果目录不存在: work_dirs/faster-rcnn_margin_default/inference_results

处理目录: work_dirs/faster-rcnn_basic_default/inference_results
推理结果目

In [2]:

# faster-rcnn_ssc_RL_wasserstein.py
# faster-rcnn_ssc_RL_kl.py
# faster-rcnn_ssc_Wasserstein.py
# faster-rcnn_ssc_Default.py
# faster-rcnn_ssc_combinatorial.py

# !python sual/inference.py \
#     custom_config/faster-rcnn_ssc_combinatorial.py \
#     --base-dir work_dirs/al_faster-rcnn_ssc_combinatorial \
#     --img-dir data/ForestDamages/val2024 \
#     --out-dir work_dirs/al_faster-rcnn_ssc_combinatorial/inference_results \
#     --device cuda:0 \
#     --score-thr 0.3 \
#     --rounds 16

# 定义配置列表

configs = [
    {
        'config': 'custom_config/faster-rcnn_ssc_combinatorial.py',
        'base_dir': 'work_dirs/al_faster-rcnn_ssc_combinatorial',
        'out_dir': 'work_dirs/al_faster-rcnn_ssc_combinatorial/inference_results'
    },
    {
        'config': 'custom_config/faster-rcnn_ssc_Default.py',
        'base_dir': 'work_dirs/al_faster-rcnn_ssc_default',
        'out_dir': 'work_dirs/al_faster-rcnn_ssc_default/inference_results'
    },

    {
        'config': 'custom_config/faster-rcnn_ssc_RL_wasserstein.py',
        'base_dir': 'work_dirs/al_faster-rcnn_ssc_rl_wasserstein',
        'out_dir': 'work_dirs/al_faster-rcnn_ssc_rl_wasserstein/inference_results'
    },
    {
        'config': 'custom_config/faster-rcnn_ssc_RL_kl.py',
        'base_dir': 'work_dirs/al_faster-rcnn_ssc_rl_kl',
        'out_dir': 'work_dirs/al_faster-rcnn_ssc_rl_kl/inference_results'
    },
    {
        "config":"custom_config/faster-rcnn_ssc_Wasserstein.py",
        "base_dir":"work_dirs/al_faster-rcnn_ssc_wasserstein",
        "out_dir":"work_dirs/al_faster-rcnn_ssc_wasserstein/inference_results"
    },
    
    {
        'config': 'custom_config/faster-rcnn_basic_default.py',
        'base_dir': 'work_dirs/faster-rcnn_basic_default',
        'out_dir': 'work_dirs/faster-rcnn_basic_default/inference_results'
    },
    {
        'config': 'custom_config/faster-rcnn_entropy.py',
        'base_dir': 'work_dirs/faster-rcnn_entropy',
        'out_dir': 'work_dirs/faster-rcnn_entropy/inference_results'
    },
    {
        'config': 'custom_config/faster-rcnn_least_confid.py',
        'base_dir': 'work_dirs/faster-rcnn_least_confid',
        'out_dir': 'work_dirs/faster-rcnn_least_confid/inference_results'
    },
    {
        'config': 'custom_config/faster-rcnn_sor.py',
        'base_dir': 'work_dirs/faster-rcnn_sor',
        'out_dir': 'work_dirs/faster-rcnn_sor/inference_results'
    },
    {
        'config': 'custom_config/faster-rcnn_margin_default.py',
        'base_dir': 'work_dirs/faster-rcnn_margin_default',
        'out_dir': 'work_dirs/faster-rcnn_margin_default/inference_results'
    },
    
]

# 遍历配置运行推理
for cfg in configs:
    print(f"\n开始处理配置: {cfg['config']}")
    cmd = f"""python sual/inference.py \
        {cfg['config']} \
        --base-dir {cfg['base_dir']} \
        --img-dir data/ForestDamages/active_learning_entropy/images_labeled_val \
        --out-dir {cfg['out_dir']} \
        --device cuda:0 \
        --score-thr 0.2 \
        --rounds 16"""
    
    print(f"执行命令: {cmd}")
    !{cmd}
    print(f"配置 {cfg['config']} 推理完成")


开始处理配置: custom_config/faster-rcnn_ssc_combinatorial.py
执行命令: python sual/inference.py         custom_config/faster-rcnn_ssc_combinatorial.py         --base-dir work_dirs/al_faster-rcnn_ssc_combinatorial         --img-dir data/ForestDamages/active_learning_entropy/images_labeled_val         --out-dir work_dirs/al_faster-rcnn_ssc_combinatorial/inference_results         --device cuda:0         --score-thr 0.2         --rounds 16
找到 16 张图片

处理第 1 轮...
Loads checkpoint by local backend from path: work_dirs/al_faster-rcnn_ssc_combinatorial/round_1/epoch_1.pth
Round 1 Processing: 100%|███████████████████████| 16/16 [00:01<00:00, 11.96it/s]
第 1 轮结果已保存到: work_dirs/al_faster-rcnn_ssc_combinatorial/inference_results/round_1/inference_results.json

处理第 2 轮...
Loads checkpoint by local backend from path: work_dirs/al_faster-rcnn_ssc_combinatorial/round_2/epoch_1.pth
Round 2 Processing: 100%|███████████████████████| 16/16 [00:00<00:00, 18.59it/s]
第 2 轮结果已保存到: work_dirs/al_faster-rcnn_ssc_combinator

#### 4.绘制各轮次teacher模型的置信度分布和置信度变化趋势

In [17]:
import json
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path

def extract_confidence_scores(json_file, conf_threshold=0.2):
    """从推理结果JSON文件中提取置信度分数"""
    with open(json_file, 'r') as f:
        results = json.load(f)
    
    all_scores = []
    for img_name, result in results.items():
        scores = result['predictions']['scores']
        filtered_scores = [score for score in scores if score >= conf_threshold]
        all_scores.extend(filtered_scores)
    
    return all_scores

def plot_confidence_distribution(all_rounds_data, save_dir, conf_threshold=0.2):
    save_dir = Path(save_dir)
    save_dir.mkdir(parents=True, exist_ok=True)  # 自动创建目录
    print(f"图片将保存至: {save_dir.absolute()}")
    """绘制所有轮次的置信度分布图"""
    plt.rcParams.update({
        'figure.figsize': (20, 20),
        'font.size': 12,
        'axes.labelsize': 14,
        'axes.titlesize': 16
    })
    
    fig, axes = plt.subplots(4, 4, figsize=(20, 20))
    axes = axes.flatten()
    
    # 计算最大频率和统计信息
    max_frequency = 0
    rounds_stats = {
        'mean_scores': [],
        'median_scores': [],
        'total_boxes': [],
        'high_conf_ratio': []
    }
    
    # 设置直方图的bins
    bin_width = 0.05
    bins = np.arange(conf_threshold, 1.05, bin_width)
    
    # 找到最大频率
    valid_rounds = 0  # 记录有效轮次数
    for scores in all_rounds_data:
        if scores:
            valid_rounds += 1
            hist_data = np.histogram(scores, bins=bins)
            max_frequency = max(max_frequency, np.max(hist_data[0]))
    
    # 绘制每轮分布图
    for round_idx, scores in enumerate(all_rounds_data, 1):
        ax = axes[round_idx - 1]
        if scores:
            n, bins, patches = ax.hist(scores, bins=bins, alpha=0.7,
                                     color='#2878B5', edgecolor='black')
            
            mean_score = np.mean(scores)
            median_score = np.median(scores)
            total_boxes = len(scores)
            high_conf_ratio = len([s for s in scores if s >= 0.5]) / len(scores)
            
            rounds_stats['mean_scores'].append(mean_score)
            rounds_stats['median_scores'].append(median_score)
            rounds_stats['total_boxes'].append(total_boxes)
            rounds_stats['high_conf_ratio'].append(high_conf_ratio)
            
            title = (f'Round {round_idx}\n'
                    f'Total Boxes: {total_boxes}\n'
                    f'Mean: {mean_score:.3f}, Median: {median_score:.3f}\n'
                    f'High Conf Ratio: {high_conf_ratio:.2%}')
            
            ax.set_title(title)
            ax.set_xlabel('Confidence Score')
            ax.set_ylabel('Frequency')
            ax.grid(True, alpha=0.3)
            
            ax.axvline(mean_score, color='#C82423', linestyle='--', alpha=0.8,
                      label=f'Mean: {mean_score:.3f}')
            ax.axvline(median_score, color='#28A428', linestyle='--', alpha=0.8,
                      label=f'Median: {median_score:.3f}')
            
            ax.legend()
            ax.set_ylim(0, max_frequency * 1.1)
        else:
            ax.set_title(f'Round {round_idx}\nNo Data')
            # 添加空数据以保持数组长度一致
            rounds_stats['mean_scores'].append(np.nan)
            rounds_stats['median_scores'].append(np.nan)
            rounds_stats['total_boxes'].append(0)
            rounds_stats['high_conf_ratio'].append(np.nan)
        
        ax.set_xlim(conf_threshold - 0.05, 1.05)
        ticks = np.arange(np.ceil(conf_threshold * 10) / 10, 1.1, 0.1)
        ax.set_xticks(ticks)
    
    main_title = f'Confidence Score Distribution (Threshold ≥ {conf_threshold})'
    plt.suptitle(main_title, fontsize=20, y=1.02)
    plt.tight_layout()
    
    # 保存分布图
    dist_plot_path = os.path.join(save_dir, f'confidence_distribution_thr_{conf_threshold}.jpg')
    print(f"分布图将保存至: {dist_plot_path}")
    plt.savefig(dist_plot_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    # 绘制趋势图
    if valid_rounds > 0:  # 只在有有效数据时绘制趋势图
        plt.figure(figsize=(15, 5))
        rounds_range = range(1, len(all_rounds_data) + 1)
        
        # 转换为numpy数组以便处理nan值
        mean_scores = np.array(rounds_stats['mean_scores'])
        high_conf_ratio = np.array(rounds_stats['high_conf_ratio'])
        
        # 只绘制非nan值
        valid_mask = ~np.isnan(mean_scores)
        plt.plot(np.array(list(rounds_range))[valid_mask], 
                mean_scores[valid_mask],
                label='Mean Confidence', marker='o', color='#C82423')
        
        valid_mask = ~np.isnan(high_conf_ratio)
        plt.plot(np.array(list(rounds_range))[valid_mask], 
                high_conf_ratio[valid_mask],
                label='High Conf Ratio (≥0.5)', marker='s', color='#28A428')
        
        plt.xlabel('Round')
        plt.ylabel('Score')
        plt.title(f'Model Performance Trend (Threshold ≥ {conf_threshold})')
        plt.legend()
        plt.grid(True)
        
        trend_plot_path = os.path.join(save_dir, f'confidence_trend_thr_{conf_threshold}.jpg')
        plt.savefig(trend_plot_path, dpi=300, bbox_inches='tight')
        plt.close()
    
    return rounds_stats




In [9]:
# 分析结果
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# 遍历work_dirs下的所有目录
work_dirs = os.listdir("work_dirs")
for work_dir in work_dirs:
    if work_dir.startswith("al_faster-rcnn_ssc_") or work_dir.startswith("faster-rcnn_"):
        inference_results_dir = f"work_dirs/{work_dir}/inference_results"
        print(f"\n处理目录: {inference_results_dir}")
        conf_threshold = 0.2

        # 收集所有轮次的数据
        all_rounds_data = []
        for round_idx in range(1, 17):
            json_file = os.path.join(inference_results_dir, f'round_{round_idx}', 'inference_results.json')
            if os.path.exists(json_file):
                round_scores = extract_confidence_scores(json_file, conf_threshold)
                all_rounds_data.append(round_scores)
                # print(f"Round {round_idx}: {len(round_scores)} detections above threshold {conf_threshold}")
            else:
                # print(f"Round {round_idx}: No results file found")
                all_rounds_data.append([])

        # 绘制分布图并获取统计信息
        stats = plot_confidence_distribution(all_rounds_data, 
                                          save_dir=inference_results_dir,
                                          conf_threshold=conf_threshold)

        # # 打印统计信息
        # print("\n统计信息概要：")
        # for round_idx, (mean_score, high_ratio) in enumerate(zip(stats['mean_scores'], 
        #                                                        stats['high_conf_ratio']), 1):
        #     if not np.isnan(mean_score):
        #         print(f"Round {round_idx}:")
        #         print(f"  平均置信度: {mean_score:.3f}")
        #         print(f"  高置信度比例: {high_ratio:.2%}")


处理目录: work_dirs/al_faster-rcnn_ssc_rl_kl/inference_results
图片将保存至: /data/22_huangguobin/mmdetection-3.1.0/work_dirs/al_faster-rcnn_ssc_rl_kl/inference_results
分布图将保存至: work_dirs/al_faster-rcnn_ssc_rl_kl/inference_results/confidence_distribution_thr_0.2.jpg

处理目录: work_dirs/al_faster-rcnn_ssc_rl_wasserstein/inference_results
图片将保存至: /data/22_huangguobin/mmdetection-3.1.0/work_dirs/al_faster-rcnn_ssc_rl_wasserstein/inference_results
分布图将保存至: work_dirs/al_faster-rcnn_ssc_rl_wasserstein/inference_results/confidence_distribution_thr_0.2.jpg

处理目录: work_dirs/al_faster-rcnn_ssc_wasserstein/inference_results
图片将保存至: /data/22_huangguobin/mmdetection-3.1.0/work_dirs/al_faster-rcnn_ssc_wasserstein/inference_results
分布图将保存至: work_dirs/al_faster-rcnn_ssc_wasserstein/inference_results/confidence_distribution_thr_0.2.jpg

处理目录: work_dirs/al_faster-rcnn_ssc_combinatorial/inference_results
图片将保存至: /data/22_huangguobin/mmdetection-3.1.0/work_dirs/al_faster-rcnn_ssc_combinatorial/inference_results
分布图将保

#### 6.各个实验置信度变化趋势

In [15]:
import json
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path

def extract_confidence_scores(json_file, conf_threshold=0.4):
    """
    从推理结果JSON文件中提取置信度分数
    """
    try:
        with open(json_file, 'r') as f:
            results = json.load(f)
        
        all_scores = []
        # 遍历每个图片的结果
        for img_result in results.values():
            if isinstance(img_result, dict) and 'predictions' in img_result:
                scores = img_result['predictions'].get('scores', [])
                filtered_scores = [score for score in scores if score >= conf_threshold]
                all_scores.extend(filtered_scores)
        return all_scores
    except Exception as e:
        print(f"Error processing {json_file}: {str(e)}")
        return []

def compare_confidence_trends(base_dir='work_dirs', save_path='confidence_comparison.jpg'):
    """
    比较不同实验的置信度趋势，使用更专业的配色和线型
    """
    # 实验名称和对应的显示标签
    # 实验名称和对应的显示标签
    experiment_labels = {
        # 'al_faster-rcnn_ssc_combinatorial': 'SSC Combinatorial',
        # 'al_faster-rcnn_ssc_default': 'SSC *',
        # 'al_faster-rcnn_ssc_rl_kl': 'SSC RL-KL',
        # 'al_faster-rcnn_ssc_wasserstein': 'SSC Wasserstein',
        # "al_faster-rcnn_ssc_rl_wasserstein": "SSC RL-Wasserstein",
        "faster-rcnn_entropy": "Entropy *",
        "faster-rcnn_least_confid": "Least Confidence *",
        "faster-rcnn_sor": "SOR *",
        "faster-rcnn_margin_default": "Margin *",
        "faster-rcnn_basic_default": "Basic *"
    }

    # 专业的配色方案 - 所有颜色唯一且视觉可区分
    colors = {
        # 'SSC Combinatorial': '#992224',    # 深红色
        # 'SSC *': '#4DAF4A',               # 翠绿色
        # 'SSC RL-KL': '#7895C1',           # 柔和蓝
        # 'SSC Wasserstein': '#8074C8',     # 紫罗兰
        # 'SSC RL-Wasserstein': '#EFBB67',  # 暖橙色
        'Entropy *': '#A8CBDF',           # 浅蓝
        'Least Confidence *': '#E3625D',  # 珊瑚红
        'SOR *': '#B54764',              # 深玫红
        'Margin *': '#5C7A29',           # 橄榄绿 (新增)
        'Basic *': '#E78AC3'             # 粉红色 (新增)
    }

    # 不同的线型和标记 - 修正了拼写错误并确保多样性
    styles = {
        # 'SSC Combinatorial': {'linestyle': '-', 'marker': 'o'},
        # 'SSC *': {'linestyle': '--', 'marker': 's'},
        # 'SSC RL-KL': {'linestyle': '-.', 'marker': '^'},
        # 'SSC Wasserstein': {'linestyle': ':', 'marker': 'D'},
        # 'SSC RL-Wasserstein': {'linestyle': '-', 'marker': 'v'},
        'Entropy *': {'linestyle': '-', 'marker': 'o'},
        'Least Confidence *': {'linestyle': '-', 'marker': 's'},
        'SOR *': {'linestyle': '-', 'marker': 'D'},
        'Margin *': {'linestyle': '--', 'marker': 'p'},  # 五边形标记
        'Basic *': {'linestyle': ':', 'marker': '*'}
    }
    
    # 使用matplotlib内置样式
    plt.style.use('classic')
    fig = plt.figure(figsize=(10, 6))  # 调整图形大小以适应右侧图例
    ax = fig.add_subplot(111)
    
    # 设置背景色和网格
    ax.set_facecolor('#f8f9fa')
    fig.patch.set_facecolor('white')
    ax.grid(True, linestyle='--', alpha=0.3, color='gray')
    
    # 遍历每个实验
    for exp_dir in os.listdir(base_dir):
        if exp_dir in experiment_labels:
            inference_dir = os.path.join(base_dir, exp_dir, 'inference_results')
            mean_scores = []
            
            # 收集每轮的平均置信度
            for round_idx in range(1, 17):
                json_file = os.path.join(inference_dir, f'round_{round_idx}', 'inference_results.json')
                if os.path.exists(json_file):
                    scores = extract_confidence_scores(json_file, conf_threshold=0.4)
                    if scores:
                        mean_score = np.mean(scores)
                        mean_scores.append(mean_score)
                    else:
                        mean_scores.append(np.nan)
                else:
                    mean_scores.append(np.nan)
            
            # 绘制趋势线
            rounds = range(1, len(mean_scores) + 1)
            valid_mask = ~np.isnan(mean_scores)
            label = experiment_labels[exp_dir]
            
            ax.plot(np.array(list(rounds))[valid_mask], 
                   np.array(mean_scores)[valid_mask],
                   label=label,
                   color=colors[label],
                   linestyle=styles[label]['linestyle'],
                   marker=styles[label]['marker'],
                   linewidth=2.5,
                   markersize=8,
                   markerfacecolor='white',
                   markeredgewidth=2,
                   markeredgecolor=colors[label])
    
    # 优化图表样式
    ax.set_xlabel('Active Learning Round', fontsize=13, fontweight='bold')
    ax.set_ylabel('Mean Confidence Score', fontsize=13, fontweight='bold')
    ax.set_title('Comparison of Mean Confidence Scores\nAcross Different Methods', 
                fontsize=16, fontweight='bold', pad=20)
    
    # 优化图例位置和样式
    legend = ax.legend(fontsize=11, 
                      frameon=True,
                      facecolor='white',
                      edgecolor='lightgray',
                      loc='upper right',  # 改为右上角
                      bbox_to_anchor=(1, 1),
                        ncol=2)  # 微调位置
    
    # 设置坐标轴范围和刻度
    ax.set_ylim(0.3, 1.0)
    ax.set_xlim(0.5, 16.5)
    ax.set_xticks(range(1, 17))
    ax.set_yticks(np.arange(0.3, 1.1, 0.1))
    
    # 设置刻度标签字体大小
    ax.tick_params(axis='both', labelsize=11)
    
    # 移除上边框和右边框
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_linewidth(0.5)
    ax.spines['bottom'].set_linewidth(0.5)
    
    # 保存图片，确保图例不被裁剪
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight', pad_inches=0.2)
    plt.close()
    
    print(f"比较图已保存至: {save_path}")

# 调用函数
compare_confidence_trends()

/tmp/ipykernel_40961/4252195862.py:127: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  legend = ax.legend(fontsize=11,


比较图已保存至: confidence_comparison.jpg


### 批量测试验证集

In [ ]:
!python sual/batch_test.py